# Introduction
This tutorial will give an example application of using deep learning for medical image-to-image translation. This example will demonstrate how to transform a segmentation CNN into a regression CNN for the purpose of predicting T2 images from T1 images. 

Keep an eye out for questions through this demo to test your new DL knowledge and critical thinking. There are answers at the end of the document.

### Initial preparation

Import necessary modules

In [3]:
import os # operating system operations 
import numpy as np # number crunching
np.random.seed(1) # set seed for random number generator
import keras # our deep learning library
import matplotlib.pyplot as plt # for plotting our results
# set plotting to be in-line and interactive
%matplotlib notebook

Using TensorFlow backend.


We will import other necessary modules as we go and need them

# Data Preparation
All deep learning applications start with getting the data.

We made this one easy for you by compiling the data into an HDF5 file. All we have to do is load all of the inputs and targets and it will be ready to go.

First we import the python hdf5 library, h5py. Then we load all the individual datasets in and convert them to Numpy arrays.

In [5]:
# load training, validation, and testing data
import h5py
with h5py.File('ImageTranslationData.hdf5','r') as hf:
    trainX = np.array(hf.get("trainX"))
    trainY = np.array(hf.get("trainY"))
    valX = np.array(hf.get("valX"))
    valY = np.array(hf.get("valY"))
    testX = np.array(hf.get("testX"))
    testY = np.array(hf.get("testY"))

This time, we will use an ImageDataGenerator so we can augment data on the fly. Before, we used this on a directory to make loading in image data from directories a breeze. However, that only works for classification schemes. For this image to image translation problem, we first had to load all the data into an array. Already did it!

Next, we have to setup two generators, one for the input images and one for the target images. Then we will synchronize the generators so they are always creating augmented inputs and targets that match up for training.

We have already set these up for you, but it is helpful to look through all the different parameters that are set in the augmentation.

In [ ]:
 # setup image data generator
datagen1 = ImageDataGenerator(
    rotation_range=15,
    shear_range=0.5,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest')
datagen2 = ImageDataGenerator(
    rotation_range=15,
    shear_range=0.5,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest')

# Provide the same seed and keyword arguments to the fit and flow
# in order to synchronize the inputs and targets
seed = 1
datagen1.fit(trainX, seed=seed)
datagen2.fit(trainY, seed=seed)
batchsize = 16
datagen = zip(datagen1.flow( trainX, None, batchsize, seed=seed),
              datagen2.flow( trainY, None, batchsize, seed=seed))

# calculate number of epochs and batches
numEp = 10
steps = np.minimum(np.int(trainX.shape[0]/batchsize*16),1000)
numSteps = steps*numEp

Now all you have to do is call your compiled model on this data generator. Here's the syntax:

`hist = Model.fit_generator(DataGenerator, steps_per_epoch,epochs,validation_data=(x,y)`

Fill in the syntax with the parameters calculated above, and let it run

In [ ]:
RegModel.fit_generator(datagen,
                     steps_per_epoch=steps,
                     epochs=numEp,
                     validation_data=(valX,valY))

It's always a good to check that our data loaded correctly and the inputs correspond to the target images.

In [ ]:
disp_ind = 50
plt.figure()
disp = np.c_[inputs[disp_ind,...,0],
             targets[disp_ind,...,0]]
plt.imshow(disp,cmap='gray')
plt.show()

Looks good!

# Transforming Segmentation Network into Translation Network

We have already built a segmentation CNN in the convolutional encoder-decoder format. This is exactly what we need for image to image translation since our targets are equivalent in size to our inputs.

However, there are some changes we need to make. Our output is continuous rather than binary. This means:
1. We want a continuous output that isn't squashed by a sigmoid function
2. We need a loss function that works with regression error, not overlap error like Dice

We will start with the previously made network that has skip connections and had good performance on the segmentation task. 

Let's get to work!

We have already imported keras, so we don't technically need to import anything. It keep code a lot cleaner to individually import layers, so we'll do that again.

In [ ]:
# import keras layers
from keras.layers import Input,Conv2D,ZeroPadding2D
from keras.layers import concatenate,Conv2DTranspose
from keras.models import Model

Below is the code used to define the segmentation network in the previous example, including the use of skip connections. Make the appropriate alterations to this code as described above to transform it into a image-to-image translation model.

In [ ]:
inp = Input(shape=x_train.shape[1:])
x = Conv2D(10,kernel_size=(3,3),padding='same',activation='relu',kernel_initializer=init)(inp)
x1 = Conv2D(20, kernel_size=(3,3),padding='same',activation='relu',kernel_initializer=init)(x)
zp = ZeroPadding2D(padding=(1,1))(x1)
x = Conv2D(30, kernel_size=(4,4),
                strides=(2,2),
                activation='relu',
                kernel_initializer=init)(zp)
x = Conv2D(30, kernel_size=(3,3),padding='same',activation='relu',kernel_initializer=init)(x)
x2 = Conv2D(40, kernel_size=(3,3),padding='same',activation='relu',kernel_initializer=init)(x)
zp = ZeroPadding2D(padding=(1,1))(x2)
x = Conv2D(40, kernel_size=(4,4),
                strides=(2,2),
                activation='relu',
                kernel_initializer=init)(zp)
x = Conv2D(50, kernel_size=(3,3),padding='same',activation='relu',kernel_initializer=init)(x)
x = Conv2D(50, kernel_size=(3,3),padding='same',activation='relu',kernel_initializer=init)(x)
x = Conv2DTranspose(40, kernel_size=(4,4),
                        strides=(2,2),
                        activation='relu',
                        kernel_initializer=init)(x)
x = Conv2D(40, kernel_size=(3,3),activation='relu',kernel_initializer=init)(x)
x = concatenate([x,x2])
x = Conv2D(30, kernel_size=(3,3),padding='same',activation='relu',kernel_initializer=init)(x)
x = Conv2D(30, kernel_size=(3,3),padding='same',activation='relu',kernel_initializer=init)(x)
x = Conv2DTranspose(20, kernel_size=(4,4),
                        strides=(2,2),
                        activation='relu',
                        kernel_initializer=init)(x)
x = Conv2D(20, kernel_size=(3,3),activation='relu',kernel_initializer=init)(x)
x = concatenate([x,x1])
x = Conv2D(10, kernel_size=(3,3),padding='same',activation='relu',kernel_initializer=init)(x)
x = Conv2D(10, kernel_size=(3,3),padding='same',activation='relu',kernel_initializer=init)(x)

# Final output layer
out = Conv2D(1,kernel_size=(1,1),activation='linear',kernel_initializer=init)(x)

RegModel = Model(inp,out)

Next, define the loss function you wish to use for this problem

In [ ]:
loss = keras.losses.mean_absolute_error

Finally, add an optimizer and compile the model

In [ ]:
opt = keras.optimizers.Adam()

SegModel.compile(loss=loss,optimizer=opt)

### Model Training
Let's try training!

In [ ]:
# Run the model training with our x and y training data, batch size of 16,
# 5 epochs, shuffle on, and provide our validation data
# Save the output to the variable 'hist'
hist = RegModel.fit(x_train, y_train,
          batch_size=32,
          epochs=5,
          verbose=1,
          shuffle=True,
          validation_data=(x_val, y_val))

### Evaluate Model
After the training is complete, we evaluate the model again on our validation data to see the results.

In [ ]:
# Get the Dice score from evaluating the model and print it out
score = RegModel.evaluate(x_val, y_val, verbose=0)
print('Final MAE on validation set: {:.04e}'.format(score))

Plot the loss curves too:

In [ ]:
# Plot the losses that are stored in the 'hist' variable
plt.figure(figsize=(6.0, 4.0));
plt.plot(hist.epoch,hist.history['loss'],'b-s')
plt.plot(hist.epoch,hist.history['val_loss'],'r-s')
plt.legend(['Training Loss',
            ' Validation Loss'])
plt.xlabel('Epochs')
plt.ylabel('Dice Loss')
plt.ylim([0,1])
plt.show()

An important thing to look for is that the validation loss isn't increasing while the training loss decreases. The divergence of the losses like this means that the model is overfitting- it is getting really good at the training data that it sees, but it is getting worse at the data that it doesn't see. This means the model won't be very helpful when we want to apply it to new data.
Due to the random initialization of the network, the exact loss plots will be different every single time you train it. However, for this example, some general statements can be made that probably apply to your results.
* The validation and training losses generally go down. This is good- the model is learning.
* The validation loss is generally higher than the training loss. This is expected- the model will learn the training data best because that is what it gets direct feedback on. The hope is that it will transfer what it learns to the validation data too.
* The validation loss spikes up at some point. This is also pretty normal. The validation data isn't part of the feedback loop so it's not guaranteed that the model will consistently get better results on it. As long as the spikes are isolated and the validation loss follows a general downward trend, it's not anything to worry about.

##### Question 3: What techniques or strategies can be used to mitigate issues with overfitting?

Another useful way to evaluate a model is to just look at the outputs. We can look at a sample image to see how the mask looks compared to the ground truth.

In [ ]:
# Get the predictions of the model on the validation inputs
predictions = SegModel.predict(x_val)

In [ ]:
# pick a random slice to examine
disp_ind = 45
# get the CT image, the model predicted mask, and the target mask
image = x_val[disp_ind,...,0]
predicted_mask = predictions[disp_ind,...,0]
truth_mask = y_val[disp_ind,...,0]
# normalize image for display
image = image-np.min(image)
image = image/np.max(image)
# create a figure
plt.figure()
# combine images together into one
disp = np.c_[image,predicted_mask,truth_mask]
# display image
plt.imshow(disp,cmap='gray')
plt.show()

Results will vary here. It's unlikely that the model already learned a beautiful mask, but hopefully it at least learned something useful and can produce a somewhat reasonable result.

Sometimes it helps to get more precise visualization. We have provided a function for viewing the mask on top of the image, so we can maybe start to explain what mistakes the model is making.

In [ ]:
from Demo_Functions import display_mask

display_mask(image,predicted_mask)

##### Question 4: Can you explain the errors made by the deep learning model?

There are a variety of directions to go from here

A deeper net gives more representational power to the model. If the problem is too complex for the current network, making it deeper should improve performance.

Some mathematical tricks, like batch normalization and ELU activations can help with the learning process and make the model learn quicker.

Deep learning models are generally trained for much longer than how long we are running for this example.

In segmentation, a particularly useful trick is the use of skip connetions, in which layers from the downsampling part of the network are concatenated with layers on the upsampling part. This both boosts the representational power of the model as well as improves the gradient flow, which also helps the model learn quicker.
However, these skip connections take a little bit more effect to implement. Luckily, Keras still makes it pretty easy.

## Part 3: Functional API
So far, we've been making sequential models.
Basically, it means that our network
has a single, straight path, i.e.

![Simple CNN floatchart](https://github.com/jmj23/deep-learning/raw/master/BootCamp/CNN_simple_flowchart.png "Simple CNN")

Each layer has a single input and output

But what if we wanted something more complicated? What if
we wanted to implement the skip connections that were just mentioned, for example? Then we would want something like

![Connection CNN floatchart](https://github.com/jmj23/deep-learning/raw/master/BootCamp/CNN_connection_flowchart.png "Connection CNN")

               
The extra connection shown is called a skip connection. Skip connections allow the model to consider features that were calculated earlier in the network again, merged with further processed features in practice, this has shown to be hugely helpful in geting precise localization in segmentation outputs.

We'll use the same segmentation data so no need to prepare anything new. Let's jump into model creation.

## Build a segmentation model with skip connections

In [ ]:
# A new layer we will need for this model
from keras.layers import concatenate

# start like before
inp = Input(shape=x_train.shape[1:])
# add on a couple convolutional layers
# We don't need to keep track of every layer- just
# a few of them. We won't keep track of the first one
# but we'll keep the second one and name it x1
x = Conv2D(10,kernel_size=(3,3),padding='same',activation='relu',kernel_initializer=init)(inp)
x1 = Conv2D(20, kernel_size=(3,3),padding='same',activation='relu',kernel_initializer=init)(x)
# We will have to use some specific zero padding
# to keep our layer sizes friendly for skip connections
# import 2D zero padding from keras layers
from keras.layers import ZeroPadding2D
# make a zero padding layer that does 1 pad of zeros
# on all sides
zp = ZeroPadding2D(padding=(1,1))(x1)
# Add a strided convolution layer
x = Conv2D(30, kernel_size=(4,4),
                strides=(2,2),
                activation='relu',
                kernel_initializer=init)(zp)
# Now repeat the process, hanging onto the second layer again
x = Conv2D(30, kernel_size=(3,3),padding='same',activation='relu',kernel_initializer=init)(x)
x2 = Conv2D(40, kernel_size=(3,3),padding='same',activation='relu',kernel_initializer=init)(x)
zp = ZeroPadding2D(padding=(1,1))(x2)
x = Conv2D(40, kernel_size=(4,4),
                strides=(2,2),
                activation='relu',
                kernel_initializer=init)(zp)
# We've now done 2 downsampling layers, like before.
# Now for the decoding side of the network, we will start
# adding skip connections
# The first couple of layers are the same as usual.
x = Conv2D(50, kernel_size=(3,3),padding='same',activation='relu',kernel_initializer=init)(x)
x = Conv2D(50, kernel_size=(3,3),padding='same',activation='relu',kernel_initializer=init)(x)
# Now our upsampling layer
x = Conv2DTranspose(40, kernel_size=(4,4),
                        strides=(2,2),
                        activation='relu',
                        kernel_initializer=init)(x)
x = Conv2D(40, kernel_size=(3,3),activation='relu',kernel_initializer=init)(x)
# This layer is now the same size as the second layer we kept.
# It can be tough to get layers to match up just right in size
# Playing around with kernel size and strides is usually needed
# so that concatenation can take place. The x,y spatial dimensions
# must be the same. Number of channels doesn't matter.
# Luckily, we already did the work for you so these layers can be
# concatenated
x = concatenate([x,x2])
# Now continue to add layers for the decoding side of the
# network, treating this merged layer like any other
x = Conv2D(30, kernel_size=(3,3),padding='same',activation='relu',kernel_initializer=init)(x)
x = Conv2D(30, kernel_size=(3,3),padding='same',activation='relu',kernel_initializer=init)(x)
x = Conv2DTranspose(20, kernel_size=(4,4),
                        strides=(2,2),
                        activation='relu',
                        kernel_initializer=init)(x)
x = Conv2D(20, kernel_size=(3,3),activation='relu',kernel_initializer=init)(x)
x = concatenate([x,x1])
x = Conv2D(10, kernel_size=(3,3),padding='same',activation='relu',kernel_initializer=init)(x)
x = Conv2D(10, kernel_size=(3,3),padding='same',activation='relu',kernel_initializer=init)(x)

# Final output layer
out = Conv2D(1,kernel_size=(1,1),activation='sigmoid',kernel_initializer=init)(x)

# Make the model using the input and output layers
# This won't work if the skip connections were not configured correctly
SegModel2 = Model(inp,out)

Let's print out a summary of the model to make sure it's what we want.
It's a little bit harder to keep track of layers in non-sequential format, but it's still a good way to make sure things look right.

In [ ]:
# Print the summary of the model
SegModel2.summary()

Now, everything else is just like the previous segmentation model. Let's try it out and see how it works!

In [ ]:
# Make same optimizer as before and compile the new model
opt = keras.optimizers.SGD(lr=0.05,decay=1e-6,momentum=.9,nesterov=True,clipnorm=0.5)
SegModel2.compile(loss=dice_coef,optimizer=opt)

In [ ]:
# Running the training with same data, batch size, and epochs as before
hist2 = SegModel2.fit(x_train, y_train,
          batch_size=32,
          epochs=7,
          verbose=1,
          shuffle=True,
          validation_data=(x_val, y_val))

In [ ]:
# Plot the results, including the previous ones
# Use different colors for the first and second model
plt.figure(figsize=(6.0, 4.0));
plt.plot(hist2.epoch,hist2.history['loss'],'r-')
plt.plot(hist2.epoch,hist2.history['val_loss'],'r-s')
plt.plot(hist.epoch,hist.history['loss'],'b-')
plt.plot(hist.epoch,hist.history['val_loss'],'b-s')
plt.legend(['Model 2 Training Loss',
            'Model 2 Validation Loss',
            'Model 1 Training Loss',
            'Model 1 Validation Loss'])
plt.xlabel('Epochs')
plt.ylabel('Dice Loss')
plt.show()

##### Question 5: How can the validation loss be lower than the training loss?

In [ ]:
# Get the predictions of the new model
predictions2 = SegModel2.predict(x_val)
# display image with mask like before
disp_ind = 45
image = x_val[disp_ind,...,0]
predicted_mask = predictions2[disp_ind,...,0]
truth_mask = y_val[disp_ind,...,0]
# normalize image for display
image = image-np.min(image)
image = image/np.max(image)
# create a figure
plt.figure()
# combine images together into one
disp = np.c_[image,predicted_mask,truth_mask]
# display image
plt.imshow(disp,cmap='gray')
plt.show()

It's better! The network learned much faster, as is apparent in the loss plots. The new model also already has better overall results. Additionally, the mask has more fine detail than the previous version without skip connections. Having these skip connections definitely make a difference. The difference becomes more pronounced for deeper networks (more layers) with more parameters and larger images.

To get a complete idea of the performance of the model, we can scroll through the entire validation set and see all the different masks it generated.

In [ ]:
from Demo_Functions import mask_viewer

images = np.copy(x_val[...,0])
masks = predictions2[...,0]
mask_viewer(images,masks)

Now that you know the functional API, you can make any graph you like, train it, and use it! Once you've mastered the syntax and conceptual understanding of how to connect layers, you are only limited by your imagination as far as what kind of network you can build.

## End of Segmentation Example. Happy deep learning!

## Answers to Questions
#### 1- What other choices could we make for input image normalization?

You can really normalize any way you want. Some other typical methods are normalizing to the ranges [0,1] or [-1,1]. Since these are CT images we could take advanteage of Hounsfield Units and scale water to be 0, air to be -1, etc. The import thing is that the data is fed into the model in a consistent way.
    
    
#### 2- What could be another use for having multiple input channels?

In MRI, multiple sequences are usually acquired. It might take some resampling of the data, but you could use multiple sequences as different channels, for example, T1, T2, and 2-point Dixon images. Including more channels in your inputs almost always results in better performance for a deep learning model.

#### 3- What techniques or strategies can be used to mitigate issues with overfitting?

The best solution is to use more data. That is rarely a possible solution in medical imaging, so there are some alternatives.
1. Use data augmentation to synthesize extra data
2. Reduce the size or complexity of the network
3. Introduce regularization. This can include dropout, batch normalization, or L1/L2 regularization

#### 4- Can you explain the errors made by the deep learning model?

No! It's really difficult to explain or understand exactly what is going on inside a CNN. There's simply too many parameters involved to be able to pick apart what each one is doing. That's why training always needs validation- it's the only way to check that our model is really learning something useful.

#### 5- How can the validation loss be lower than the training loss?

It generally isn't, because the model learns from the training data and not the validation data. Only in contrived scenarios could the model actually perform better on the validation data than training. However, sometimes you will see lower validation loss. The explanations could be:
* The model has equivalent performance on training and validation, and slight random differences make the validation loss slightly lower
* A quirk of Keras. This is how Keras evaluates losses during training:
    1. Calculate loss of each training batch during epoch
    2. Average these losses together at end of epoch. This is the epoch's training loss
    3. Calculate total validation loss at end of epoch.
    
If a model learns very quickly (frequent in the first few epochs) then the performance of the model at the end of the epoch, when it evaluates the validation data, will better than the average performance during the entire epoch.